In [1]:
import sys

sys.path.insert(1, "/Users/minhpham/Documents/Work/Python/Qiskit/New Project/FrozenYoghourt/")

from FrozenYoghourt import *
from FrozenYoghourt import *
from FrozenYoghourt.mode import *
from FrozenYoghourt.gates import *
from FrozenYoghourt.gates.single import *
from FrozenYoghourt.gates.double import *
from FrozenYoghourt.gates.multi import *
from FrozenYoghourt.maths import *
from FrozenYoghourt.quantum import *
from FrozenYoghourt.quantum.decomposition import *
from FrozenYoghourt.circuit import *
from FrozenYoghourt.visualization import *

import numpy as np
import sys

sys.path.insert(1, "/Users/minhpham/Documents/Research/laughing-umbrella/xx_synthesis/monodromy")

import monodromy

from monodromy.coordinates import unitary_to_monodromy_coordinate, monodromy_to_positive_canonical_coordinate

from matplotlib import pyplot as plt

/Users/minhpham/Documents/Research/laughing-umbrella/xx_synthesis/monodromy/lrslib-071b/lrs


KAK decomposition gives us 

$$CAN(a_1, a_2, 0) \cdot (Z_d \otimes Z_e) \cdot (XX_\beta) = (Z_r \otimes Z_s) \cdot CAN(b_1, b_2, 0) \cdot (Z_t \otimes Z_u).$$

We want to equate these parameters to each other. $\textbf{Note:}$ We need to multiply everything by 2 to match the convention of the paper. We also need to reverse the order of the circuit

We now build numerical circuit to check the correctness of the inequalities of Theorem 5.3. Note that we have $a_1+a_2 \leq \frac{\pi}{2}$, $a_1 \geq a_2$ and $0 \leq \beta \leq \frac{\pi}{4}$.

Once we have set up our left hand side circuit, we now compute the canonical coordinates for the right hand side circuit

In [2]:
def generate_random_a1a2():
    # Generate a random value for y in the range [0, pi/4]
    a2 = np.random.uniform(0, np.pi / 4)
    
    # Generate a random value for x in the range [y, pi/2 - y]
    low, high = np.sort([a2, np.pi / 2 - a2])
    a1 = np.random.uniform(low, high)
    
    if a1>np.pi/4:
        a1 = np.pi/2-a1
        a2, a1 = np.sort([a1, a2]) # Since a3 = 0.
    
    assert a1+a2 <= np.pi/2, "a1 + a2 is greater than pi/2"
    assert a1>=a2, "a2 is greater than a1"
    assert a2 >=0, "a2 is greater than 0"
    assert a1 <= np.pi/4, "a1 is greater than pi/4"
    
    return a1, a2

def canonical_coordinate(mat):
    # Compute canonical coordinate
    return monodromy_to_positive_canonical_coordinate(*unitary_to_monodromy_coordinate(mat)[:3])

In [345]:
# Left hand side circuit
Mode.toggle("n")

cause = []
fails = []
rounds = 10_000

for _ in range(rounds):

    a1, a2, d, e, beta = *generate_random_a1a2(), *np.random.uniform(0, np.pi, 2), np.random.uniform(0, np.pi/4)

    xx_lhs = NumericalCircuit(2)

    xx_lhs.rxx(2*beta)
    xx_lhs.rz([2*d, 2*e], [0, 1])
    xx_lhs.can(2*a1, 2*a2, 0)

    b1, b2, b3 = canonical_coordinate(xx_lhs.unitary)

    try:
        assert np.isclose(b3, 0), "The z coordinate does not vanish"
        assert b1+b2 >= a1+a2 - beta, "First inequality not satisfied"
        assert b1+b2 <= np.pi/2- np.abs(np.pi/2-(a1+a2+beta)), "Second inequality not satisfied"
        assert b1-b2 >= np.abs(a1-a2-beta) or np.pi/2-(b1+b2) >= np.abs(a1-a2-beta), "Third inequality not satisfied"
        assert b1-b2 <= a1-a2+beta, "Fourth inequality not satisfied"
    except:
        cause.append((a1, a2))
        fails.append((b1, b2))

NUMERICAL activated


In [344]:
len(fails)

8767

It turns out that having

$$b_1+b_2 \geq \frac{\pi}{2}-(a_1-a_2-\beta)$$ fixes everything.

In [42]:
x, y = zip(*fails)

fig, ax = plt.subplots()

ax.scatter(x, y, s = 1)

What if I construct the right hand side circuit and ask if the canonical coordinate afterwards actually agrees with the one before? This lead me to fixing the issues with the base-congruence.

In [16]:
Mode.toggle("n")

for _ in range(10_000):

    r, s, t, u, b1, b2 = *np.random.uniform(0, np.pi, 4), *generate_random_a1a2()

    xx_rhs = NumericalCircuit(2)
    xx_rhs.rz([2*r, 2*s], [0, 1])
    xx_rhs.can(2*b1, 2*b2, 0)
    xx_rhs.rz([2*t, 2*u], [0, 1])

    b1_, b2_, b3_ = canonical_coordinate(xx_rhs.unitary)

    assert np.isclose(b1, b1_), "b1 not close"
    assert np.isclose(b2, b2_), "b2 not close"
    assert np.isclose(b3, 0), "b3 not close"

NUMERICAL activated


Another shot for $XY$? Hehe here goes

In [351]:
# Left hand side circuit
Mode.toggle("n")

cause = []
fails = []
rounds = 10_000

for _ in range(rounds):

    a1, a2, d, e, beta = *generate_random_a1a2(), *np.random.uniform(
        0, np.pi, 2), np.random.uniform(0, np.pi / 4)

    xy_lhs = NumericalCircuit(2)

    xy_lhs.can(2 * a1, 2 * a2, 0)
    xy_lhs.rz([2 * d, 2 * e], [0, 1])
    xy_lhs.can(2 * beta, 2 * beta, 0)

    b1, b2, b3 = canonical_coordinate(xy_lhs.unitary)

    """First set of inequalities
    if not (np.isclose(a1 - a2, b1 - b2) \
        or np.isclose(a1 - a2, np.pi / 4 - np.abs(np.pi/4-(b1 + b2)))):
        cause.append((a1, a2))
        fails.append((b1, b2))
    """
    
    """Second set of inequalties"""
    assert (b1+b2 >= a1+a2-2*beta \
            or b1+b2 <= np.pi/2+np.abs(np.pi/2-(a1+a2+2*beta))), "Not correct"

NUMERICAL activated


In [350]:
len(fails)

0

If we let $a_1-a_2 = \frac{\pi}{2}-\left|\frac{\pi}{2}-(b_1+b_2)\right|$, then this seems to solve everything.

Now think about what Eric said about one dimension